# Bayesian Optimisation Notebook

<h4> This notebook compartmentalises the Bayesian Optimisation aspect of this project for cleanliness</h4>

In [2]:
import numpy as np
import torch
import torch.nn
import torch.optim 
from torch.utils.data import DataLoader, TensorDataset

import os
from tqdm import tqdm
import random
import glob

import optuna
import json

# my custom modules
import data_utils as du
import nn_tools as nt
import model_eval as me

os.chdir('/Users/Mak/Desktop/Imperial College London/Year Four/MSci Project/Codebase/')
DEVICE = torch.device("mps")

import importlib

importlib.reload(me)
importlib.reload(nt)
importlib.reload(du)


/opt/anaconda3/envs/msci_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'data_utils' from '/Users/Mak/Desktop/Imperial College London/Year Four/MSci Project/Codebase/data_utils.py'>

Setting the data and storage paths - to extract the raw data and store the binned, normalised data appropriately

In [3]:
normal_path = 'nn_outputs/wilson_mu_csr_cvr'
raw_path = 'raw_data/dstore_mu_csr_cvr'

number_bins = 10

In [10]:
du.file_to_normalise_4d(raw_path, normal_path, number_bins, bkg=False)

Processing File No. model_WET_mu_1411: 100%|██████████| 3570/3570 [10:07<00:00,  5.87it/s]


In [4]:
number_of_files = len(glob.glob(normal_path + '/normal_targets/*.csv'))

train_amount = int(0.7 * number_of_files)
val_amount = int(0.15 * number_of_files)

In [5]:
train_inputs, train_targets, val_test_inputs, val_test_targets, file_names = du.load_minmax_heights(normal_path, train_amount)

# =-=-=-= Splitting the remaining data into validation and testing datasets =-=-=-=
val_inputs, val_targets = val_test_inputs[:val_amount], val_test_targets[:val_amount]

test_inputs, test_targets = val_test_inputs[val_amount:], val_test_targets[val_amount:]

CSV Files: Bin Heights Already Normalised.


In [6]:
cfg = {'batch_train' : 64,
       'batch_test' : 8,
       'epochs' : 150,
       'dim': [len(train_inputs[0]), len(train_targets[0])],
       }

import math
obj_vals = []
validation_losses = []
stopping_patience = 50

def objective(trial, train_inputs=train_inputs, train_targets=train_targets,test_inputs=test_inputs, test_targets=test_targets):

    # ==== Set the seed to make sure each iteration is comparable ====
    s = 42

    torch.use_deterministic_algorithms(True)
    torch.backends.mps.deterministic = True
    torch.backends.mps.benchmark = False

    np.random.seed(s)
    torch.manual_seed(s)
    torch.mps.manual_seed(s)
    random.seed(s)

    """
    When you try and make the BO guess the bet and target_std there is conflict - setting these two to as low as possible signficantly
    attenuated the model's loss values itself.

    But then it would be repelled by the std_loss we were calculating pulling it the other way.

    So there was no objective way for the model to really actually learn to capture the variance of the data properly as
    these two processes to conflict resulting in just an impasse between the two eventually.

    So how can I actually boost the model's understanding of the variance of the data?

    """

    # ===== Bayesian Trial Parameters Section =====

    # ===== Loss Parameters =====

    def loss_function_KLD(reconstruction, x, mean, logvar, beta=0.1, target_std=1.0):
        MSE = torch.nn.functional.mse_loss(reconstruction, x, reduction='mean')

        target_logvar = torch.log(torch.tensor(target_std ** 2).to(DEVICE))
        KLD = -0.5 * torch.mean(1 + (logvar - target_logvar) 
                                    - (logvar.exp() / target_std ** 2))
        
        loss = MSE + beta * KLD
        
        return loss

    # ===== Number of Encoder and Decoder Layers Trial =====
    n_encoder_layers = trial.suggest_int('n_encoder_layers', 1, 5)
    n_decoder_layers = trial.suggest_int('n_decoder_layers', 1, 6)

    # ===== Layer Sizes =====
    encoder_layers = [trial.suggest_int(f'encoder_layer_{i}', 128, 512) for i in range(n_encoder_layers)]
    decoder_layers = [trial.suggest_int(f'decoder_layer_{i}', 128, 1024) for i in range(n_decoder_layers)]
    
    # ===== Dropout Layers =====
    dropout_enc = [trial.suggest_categorical(f'dropout_enc_{i}', [True, False]) for i in range(n_encoder_layers)]
    dropout_dec = [trial.suggest_categorical(f'dropout_dec_{i}', [True, False]) for i in range(n_decoder_layers)]

    # ===== Latent Dimensionality =====
    latent_dim = trial.suggest_int('latent_dim', 16, 128)
    
    # ===== Data Loaders =====
    load_train = DataLoader(TensorDataset(train_inputs, train_targets), batch_size=cfg['batch_train'], shuffle=True, num_workers=0)

    load_val = DataLoader(TensorDataset(val_inputs, val_targets), batch_size=cfg['batch_test'], shuffle=False, num_workers=0)

    # ===== Model Initialisation =====

    model = nt.CVAE_Opt(input_shape=cfg['dim'][0], output_shape=cfg['dim'][1], latent_dim=latent_dim, encoder_layers=encoder_layers, decoder_layers=decoder_layers,
                 dropout_enc=dropout_enc, dropout_dec=dropout_dec)
    model.to(DEVICE)

    optimiser = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

    # ====== Training Loop =====
    early_stopping_patience = 10
    best_val_loss = np.inf
    epochs_without_improvement = 0

    for epoch in range(cfg['epochs']):
        avg_loss, avg_val_loss = nt.train_epoch_early_CVAE(model, load_train, load_val, loss_function_KLD, optimiser, DEVICE)

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_without_improvement = 0

        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= early_stopping_patience:
            print(f"Early stopping triggered after {epoch + 1} epochs.")
            break
    obj_vals.append(avg_loss)
    validation_losses.append(avg_val_loss)
    return avg_loss

class EarlyStoppingCallback:
    def __init__(self, patience):
        self.patience = patience
        self.best_value = np.inf
        self.epochs_without_improvement = 0

    def __call__(self, study, trial):
        current_value = trial.value
        if current_value < self.best_value:
            self.best_value = current_value
            self.epochs_without_improvement = 0
        else:
            self.epochs_without_improvement += 1

        if self.epochs_without_improvement >= self.patience:
            print(f"Early stopping triggered after {trial.number + 1} trials.")
            study.stop()

# Create the Optuna study with early stopping
DEVICE = torch.device('mps')
early_stopping = EarlyStoppingCallback(patience=stopping_patience)
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=150, callbacks=[early_stopping])


[I 2025-04-12 11:06:40,745] A new study created in memory with name: no-name-f723c89f-6e08-4138-95b6-dcb0d667d542
[I 2025-04-12 11:08:15,486] Trial 0 finished with value: 0.0012177761062048375 and parameters: {'n_encoder_layers': 5, 'n_decoder_layers': 3, 'encoder_layer_0': 411, 'encoder_layer_1': 207, 'encoder_layer_2': 340, 'encoder_layer_3': 222, 'encoder_layer_4': 501, 'decoder_layer_0': 154, 'decoder_layer_1': 215, 'decoder_layer_2': 288, 'dropout_enc_0': False, 'dropout_enc_1': False, 'dropout_enc_2': False, 'dropout_enc_3': False, 'dropout_enc_4': True, 'dropout_dec_0': False, 'dropout_dec_1': False, 'dropout_dec_2': True, 'latent_dim': 109}. Best is trial 0 with value: 0.0012177761062048375.


Early stopping triggered after 86 epochs.


[I 2025-04-12 11:08:44,844] Trial 1 finished with value: 0.0022691140620736405 and parameters: {'n_encoder_layers': 4, 'n_decoder_layers': 4, 'encoder_layer_0': 134, 'encoder_layer_1': 213, 'encoder_layer_2': 365, 'encoder_layer_3': 341, 'decoder_layer_0': 501, 'decoder_layer_1': 567, 'decoder_layer_2': 406, 'decoder_layer_3': 646, 'dropout_enc_0': True, 'dropout_enc_1': False, 'dropout_enc_2': False, 'dropout_enc_3': True, 'dropout_dec_0': False, 'dropout_dec_1': True, 'dropout_dec_2': False, 'dropout_dec_3': False, 'latent_dim': 39}. Best is trial 0 with value: 0.0012177761062048375.


Early stopping triggered after 26 epochs.


[I 2025-04-12 11:09:04,563] Trial 2 finished with value: 0.00228436658217106 and parameters: {'n_encoder_layers': 5, 'n_decoder_layers': 6, 'encoder_layer_0': 380, 'encoder_layer_1': 442, 'encoder_layer_2': 420, 'encoder_layer_3': 436, 'encoder_layer_4': 172, 'decoder_layer_0': 988, 'decoder_layer_1': 959, 'decoder_layer_2': 172, 'decoder_layer_3': 844, 'decoder_layer_4': 524, 'decoder_layer_5': 856, 'dropout_enc_0': True, 'dropout_enc_1': False, 'dropout_enc_2': True, 'dropout_enc_3': True, 'dropout_enc_4': False, 'dropout_dec_0': True, 'dropout_dec_1': True, 'dropout_dec_2': False, 'dropout_dec_3': True, 'dropout_dec_4': False, 'dropout_dec_5': True, 'latent_dim': 25}. Best is trial 0 with value: 0.0012177761062048375.


...

Early stopping triggered after 59 epochs.


[I 2025-04-12 12:44:08,050] Trial 82 finished with value: 0.0012160115147707984 and parameters: {'n_encoder_layers': 4, 'n_decoder_layers': 3, 'encoder_layer_0': 304, 'encoder_layer_1': 245, 'encoder_layer_2': 384, 'encoder_layer_3': 390, 'decoder_layer_0': 335, 'decoder_layer_1': 787, 'decoder_layer_2': 455, 'dropout_enc_0': False, 'dropout_enc_1': False, 'dropout_enc_2': False, 'dropout_enc_3': False, 'dropout_dec_0': False, 'dropout_dec_1': False, 'dropout_dec_2': False, 'latent_dim': 108}. Best is trial 33 with value: 0.001181191988871433.


Early stopping triggered after 46 epochs.


[I 2025-04-12 12:44:42,918] Trial 83 finished with value: 0.0022811616596300154 and parameters: {'n_encoder_layers': 4, 'n_decoder_layers': 4, 'encoder_layer_0': 279, 'encoder_layer_1': 201, 'encoder_layer_2': 432, 'encoder_layer_3': 331, 'decoder_layer_0': 166, 'decoder_layer_1': 742, 'decoder_layer_2': 349, 'decoder_layer_3': 327, 'dropout_enc_0': False, 'dropout_enc_1': False, 'dropout_enc_2': False, 'dropout_enc_3': False, 'dropout_dec_0': False, 'dropout_dec_1': False, 'dropout_dec_2': False, 'dropout_dec_3': True, 'latent_dim': 96}. Best is trial 33 with value: 0.001181191988871433.


Early stopping triggered after 32 epochs.
Early stopping triggered after 84 trials.


In [7]:
model_param = study.best_trial
output_file_path = normal_path.split('/')[1] + '.json'

with open(output_file_path, 'w') as json_file:
    json.dump(model_param.params, json_file, indent=4)

print(f"Output dictionary saved to {output_file_path}")

Output dictionary saved to wilson_mu_csr_cvr.json


In [10]:
np.savetxt('loss_values.txt', np.array([obj_vals, validation_losses]).T, delimiter = ',', header = 'Training Loss, Validation Loss')